# Can we represent spectral lines with sparse tensors in PyTorch?

April 23-26, 2021  
gully

**New:** We can now infer RV with backpropagation so long as the RV falls within the `wingcut` window.

In [ ]:
import torch

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

### Make a tensor composed of $N_{lines}$ sparsely spaced spectral lines.

I have experimentally verified that this notebook works with up to 500,000 lines.  You may wish to turn off overplotting all the lines for $N_{lines}>500$.  Also, the amplitudes become unphysical and the spectrum can become negative because we are adding flux loss and not opacity.

In [ ]:
N_lines = 100_000 # Can go up to 500_000 but requires commenting out plots.

We will adopt units of "deciAngstrom", equal to one tenth of an Angstrom.  This choice will help us stick to using integers, which is needed for the sparse index approach.

In [ ]:
lam_lo = 105000
lam_hi = 110000
Delta_lam = lam_hi - lam_lo

In [ ]:
wingcut = 100

In [ ]:
lam_vec = torch.arange(lam_lo-wingcut, lam_hi+wingcut, step=1)
n_pix = len(lam_vec)

In [ ]:
n_pix

Draw $N_{lines}$ random wavelength centers $\lambda_c$, amplitudes $A$, and widths $w$ to simulate authentic spectral lines.

In [ ]:
lam_centers_native = lam_lo + Delta_lam*torch.rand(N_lines)
amplitudes = 0.01 + 0.3*torch.rand(N_lines)
widths = 4.1*torch.ones_like(amplitudes)

Stack the patches into a tensor and offset them by the line center position.

In [ ]:
del_lambda_rv = -10.123456789 # Demo

The truncated indices are fixed, and will not update based on the RV.  This allows the RV to be inferred with back-propagation.

#### Fixed wavelength indices and coordinates

In [ ]:
lambda_patch_trunc = torch.arange(-wingcut, wingcut,1).unsqueeze(1)
lambda_patches_trunc = torch.repeat_interleave(lambda_patch_trunc, N_lines, dim=1)

lam_centers_trunc = torch.trunc(lam_centers_native).to(torch.int)

lambda_segments_trunc = lambda_patches_trunc + lam_centers_trunc
indices = lambda_segments_trunc - (lam_lo - wingcut)

#### (In-between) Allow individual line centers to be "imperfect"
If you want individual spectral line positions to be allowed to float, you can allow `lam_centers_native` to become a variable (`grad=True`).  For now we assume the line center positions are known to unimpeachable precision.

In [ ]:
lambda_segments_native = lambda_patches_trunc + lam_centers_native

#### Variable wavelength indices and coordinates

In [ ]:
lambda_patches_native = lambda_segments_trunc - lam_centers_native + del_lambda_rv

In [ ]:
absorption_per_line = amplitudes*torch.exp(-0.5*lambda_patches_native**2/widths**2)

In [ ]:
plt.figure(figsize=(15, 3))
#plt.plot(lambda_segments_native/10, 1-absorption_per_line);
plt.xlabel('$\lambda \;(\AA)$')
plt.ylabel('Absorption per line')
plt.title('Individual lines vertically offset for clarity');

The lines sum together to make the net spectrum. To sum the spectrum we need to make a sparse tensor!

In [ ]:
indices = lambda_segments_trunc - (lam_lo - wingcut)

Flatten the 2D Matrix into a 1D Vector.

In [ ]:
indices_flat = indices.reshape(-1).unsqueeze(0)
values_flat = absorption_per_line.reshape(-1)

s = torch.sparse_coo_tensor(indices_flat, values_flat, (Delta_lam+wingcut*2,))

Now coalesce the spectrum, which sums the values at repeated indices.

In [ ]:
summed = s.coalesce()
summed_vec = summed.to_dense()

In [ ]:
lam_vec.shape, summed_vec.shape

In [ ]:
plt.figure(figsize=(15, 3))
plt.step(lam_vec/10, 1- summed_vec);
plt.xlabel('$\lambda \;(\AA)$')
plt.ylabel('Net Spectrum')
plt.title('Summed lines');

## Put it all together

Let's define a model that takes in RV, width, and optionally amplitudes.

In [ ]:
def model_spectrum(rv, width, amplitudes):
    '''Generate the model spectrum given an RV and width  
    
    parameters
    ----------
    rv: (torch float)
        The radial velocity expressed as a wavelength shift, i.e. deciAngstroms.
    width: (torch float)
        The linewidth in units of deciAngstroms.
    amplitudes: (torch tensor)
        The vector of N_lines amplitudes for the strength of each line.
    '''
    lambda_patches_native = lambda_segments_trunc - lam_centers_native + rv
    absorption_per_line = amplitudes*torch.exp(-0.5*lambda_patches_native**2/width**2)
    
    values_flat = absorption_per_line.reshape(-1)

    s = torch.sparse_coo_tensor(indices_flat, values_flat, (n_pix,))
    return s.coalesce().to_dense()

In [ ]:
plt.figure(figsize=(15, 3))
plt.step(lam_vec/10, 1- model_spectrum(10.4, 10.6, amplitudes));
plt.xlabel('$\lambda \;(\AA)$')
plt.ylabel('Net Spectrum')
plt.title('Summed lines');

In [ ]:
%%time 
model_spectrum(-31.1, 1.3, amplitudes)